In [ ]:
cmd <- function(bash_command) {
    output <- system(bash_command, intern=TRUE)
    last_line = ""
    
    for (line in output) { 
        cat(line)
        cat("\n")
        last_line = line 
    }
    
    return(last_line) 
}

In [ ]:
cmd("pip install 'sagemaker[local]' --upgrade")

In [ ]:
s3.bucket <- "<insert S3 bucket name>"
s3.prefix <- "<insert prefix>"

In [ ]:
training.s3_input_location <- paste0('s3://', s3.bucket, '/', s3.prefix, '/input/training_data.csv')
print(training.s3_input_location)

training.s3_output_location <- paste0('s3://', s3.bucket, '/', s3.prefix, '/output/custom/')
print(training.s3_output_location)

In [ ]:
library('reticulate')

sagemaker <- import('sagemaker')

In [ ]:
sagemaker[['__version__']]

In [ ]:
container <- "<insert container image URI>"

In [ ]:
role <- sagemaker$get_execution_role()

In [ ]:
LocalSession <- sagemaker$local$LocalSession

session <- LocalSession()
session$config <- list(local=list(local_code=TRUE))

In [ ]:
TrainingInput <- sagemaker$inputs$TrainingInput

sagemaker.train_input <- TrainingInput(training.s3_input_location, content_type="text/csv")

In [ ]:
Estimator <- sagemaker$estimator$Estimator

estimator <- Estimator(
    container,
    role, 
    instance_count=1L, 
    instance_type="local",
    output_path=training.s3_output_location,
    sagemaker_session=session)

In [ ]:
estimator$set_hyperparameters(a=1L, b=2L, c=3L)

In [ ]:
estimator$fit(list(train = sagemaker.train_input))

In [ ]:
estimator$model_data

In [ ]:
predictor <- estimator$deploy(
    initial_instance_count=1L,
    instance_type="local",
    endpoint_name="custom-local-r-endpoint"
)

In [ ]:
predictor$predict("42")

In [ ]:
predictor$delete_endpoint()